# Chute
This notebook is focused on analyzing the freeroll and chute turn to help drivers improve their performance.

### Reference Track
Here it is worth taking some care to select a good reference track, and possibly pick a new reference each week as your line improves. A higher quality reference track will allow you to collect higher quality feedback.

In [ ]:
%matplotlib widget
import json
import os
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from collections import defaultdict
from glob import glob
from scipy.interpolate import splprep, splev, BSpline, splrep
from scipy.optimize import minimize

from plot_utils import *
from constants import *

import reference

reference_distance, reference_track = reference.load(f'{DATA_DIR}/parsed/yyyy-mm-dd_hhmmss_SonOfThunderpuppy_0.csv')
print(f'Loaded {len(reference_distance)} reference points from {reference_track.data_path}')

In [ ]:
from track import Track
# Load track data and list all the processed files
file_patterns = [
    f'{DATA_DIR}/parsed/yyyy-mm-dd*',
]
data_files = [file for pattern in file_patterns for file in glob(pattern)]
data = []
for data_file in data_files:
    label = os.path.splitext(os.path.basename(data_file))[0]
    data.append(Track(
        label, data_file, reference_track,
    ))
    print(f'Loaded {label}')

In [ ]:
fig = plt.figure()
reference = reference_track
x0 = reference.pos_x(reference.fine_splits)
y0 = reference.pos_y(reference.fine_splits)
# transpose and flip the x coordinate to point left of the forward vector
left_x = -reference.vel_y(reference.fine_splits)
left_y = reference.vel_x(reference.fine_splits)
norm = np.sqrt(left_x**2 + left_y**2)
ref_speed = reference.speed(reference.fine_splits)

curvature = plt.subplot(3,2,1)
plt.setp(curvature.get_xticklabels(), visible=False)
deviation = plt.subplot(3,2,3, sharex=curvature)
plt.setp(deviation.get_xticklabels(), visible=False)
speed = plt.subplot(3,2,5, sharex=curvature)
birds_eye = plt.subplot(1,2,2)
for track in data:
    line, = curvature.plot(reference_distance, track.curvature_2d(track.fine_splits))
    color = line.get_color()
    
    t = track.fine_splits
    deviation_left = (track.pos_x(t) - x0) * left_x + (track.pos_y(t) - y0) * left_y
    deviation_left /= norm
    deviation.plot(reference_distance, deviation_left)
    
    label = f'{track.buggy_name}\n{track.date} roll {track.roll_number}'
    x, y = track.pos_x(track.fine_splits), track.pos_y(track.fine_splits)
    birds_eye.plot(x, y, c=color, ls='--')
    birds_eye.plot(track.full_dataframe[X].to_numpy(), track.full_dataframe[Y].to_numpy(), label=label)
    
    speed.plot(reference_distance, track.speed(t) - ref_speed, label=track)
plt.suptitle('Variable vs. Reference Distance (m)')
curvature.set_ylabel('Curvature (1/m)')
curvature.axhline(y=0, color='gray', lw=0.8)
deviation.set_ylabel('Deviation Left\n(m)')
deviation.axhline(y=0, color='gray', lw=0.8)
speed.set_ylabel('Relative Speed\n(m/s)')
speed.set_xlabel('Reference Distance (m)')
speed.axhline(y=0, color='gray', lw=0.8)
add_birds_eye_view(birds_eye, reference_track.split_xs, reference_track.split_ys, reference_distance)
add_crosshairs(deviation, speed, curvature)
birds_eye.legend()
plt.tight_layout(h_pad=0, pad=1)
plt.show()

# Next Steps